# CSV rev16 a rev21

Este notebook coge el csv de las clasificaciones de zooniverse de las revisiones 16 a 21 y con las restricciones estudiadas crea un nuevo csv solo con los parametros necesitados (path y class).

Restricciones:
    - Si retirement = nothinghere and n.specific >= 3 and n.n2 >= 6.25 -> al dataset
    - else retirement != nothing here): n.specific >5 and n.n2 >= 6.25 -> al dataset


In [1]:
# Librerias

import os

import numpy as np

In [6]:
# Constantes

Folder = "../zooClasificaciones_rev1-15_16-21/"

CSV = Folder + "zooClasificaciones_rev16_21.csv"

NEW_CSV = Folder + "zooClasificaciones_rev16_21_procesadas.csv"

assert (os.path.isfile(CSV)), "El csv introducido no existe"


In [11]:
# Extraer datos necesarios del csv (foto_id, classification, retirement, n.specific, howmanymeadian, n.n2)


#                                       FUNCION comprobarFechasImagenes

# leer excel y obtiene los datos de las fechas por camara
# Devuelve:
#   - foto_id
#   - classification
#   - retirement
#   - n.specific
#   - n.n2

def leerCSV(inputcsv = CSV):
    # lee el csv a procesar

    # los datos que obtiene del csv
    foto_id, classification, retirement, n_specific, n_n2 = np.loadtxt(inputcsv, dtype=str,
                                     delimiter=',', usecols=(0, 1, 2, 3, 5), unpack=True)

    foto_id = [foto.replace("\"","") for foto in foto_id]
    classification = [classe.replace("\"","") for classe in classification]

    retirement = [ret.replace("\"","") for ret in retirement]

    return foto_id, classification, retirement, n_specific, n_n2


def get_indexes(foto_id, retirement, n_specific, n_n2):
    indexes = []
    # indexes.append(0) # cabecera

    # Para cada item de la lista de fotos guardadas cuando se leyo el csv, coge el indice y la foto individual
    for index, foto in enumerate(foto_id):

        # foto_id[index] = foto_id[index].replace("\"","")

        if index != 0:

            # primero concatenar para que el acceso sea desde la carpeta procesadas
            foto_id[index] = "procesadas/" + foto_id[index]

            # caso Si retirement = nothinghere and n.specific >= 3 and n.n2 >= 6.25 -> al dataset
            if retirement[index] == "nothing_here"  and int(n_specific[index]) >= 3 and float(n_n2[index]) >= 0.625:
                indexes.append(index)
            # else retirement != nothing here): n.specific >5 and n.n2 >= 6.25 -> al dataset
            elif int(n_specific[index]) >= 5 and float(n_n2[index]) >= 0.625:
                indexes.append(index)

    return indexes

def write_csv(data, indexes_list, new_csv):
    assert indexes_list, "Lista de indices vacia, cuidado!"

    csvArray = np.empty([len(indexes_list), 2], dtype=object)

    for idx, index in enumerate(indexes_list):
        csvArray[idx] = data[index]

    if os.path.exists(new_csv):
        os.remove(new_csv)

    np.savetxt(new_csv, csvArray, delimiter=",",fmt='%s')

In [12]:
# leer csv
foto_id, classification, retirement, n_specific, n_n2 = leerCSV(CSV)

indexes = get_indexes(foto_id, retirement, n_specific, n_n2)

data = np.column_stack([foto_id,classification])

In [13]:
data

array([['foto_ID', 'classification'],
       ['procesadas/rev16/10_16/10_20220126 (1).JPG', 'HUMANORVEHICLE'],
       ['procesadas/rev16/10_16/10_20220126 (100).JPG', 'REDDEER'],
       ...,
       ['procesadas/rev22/8/8_20221125 (147).JPG', 'NOANIMAL'],
       ['procesadas/rev22/9/9_20220918 (448).JPG', 'NOANIMAL'],
       ['procesadas/rev22/9/9_20220919 (455).JPG', 'NOANIMAL']],
      dtype='<U45')

In [14]:
write_csv(data, indexes, NEW_CSV)